## Model: Whisper

In [1]:
!pip install -U openai-whisper --quiet
!apt-get install ffmpeg --quiet

Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import whisper
import os

# Load Whisper model
model = whisper.load_model("small")

# Paths
audio_folder = "/content/drive/MyDrive/audio_files/"
output_folder = "/content/drive/MyDrive/ServiceNow_Audio_Transcripts/"
os.makedirs(output_folder, exist_ok=True)

# Transcribe only if transcript file doesn't exist
for filename in os.listdir(audio_folder):
    if filename.endswith((".mp3", ".wav", ".m4a")):
        base_name = os.path.splitext(filename)[0]
        transcript_path = os.path.join(output_folder, base_name + ".txt")

        if os.path.exists(transcript_path) and os.path.getsize(transcript_path) > 0:
            print(f"Transcript already exists for: {filename} — Skipping.")
            continue

        print(f"Transcribing: {filename}")
        audio_path = os.path.join(audio_folder, filename)
        result = model.transcribe(audio_path)

        with open(transcript_path, "w", encoding='utf-8') as f:
            f.write(result['text'])

Transcript already exists for: ThW6lPyYgYk.mp3 — Skipping.
Transcript already exists for: XUKTyE2YtHc&list=PLh-u-epknspBswAAKG0EfPHyV6gcVVOhK.mp3 — Skipping.
Transcript already exists for: Rx65d0ofz8I.mp3 — Skipping.
Transcript already exists for: WyQTP0AA1VU.mp3 — Skipping.
Transcript already exists for: ZYJqkxGrNiI.mp3 — Skipping.
Transcript already exists for: E6m8UuVhIzw.mp3 — Skipping.
Transcript already exists for: K6z4c256gzI.mp3 — Skipping.
Transcript already exists for: KSWNDuKn9t0.mp3 — Skipping.
Transcript already exists for: 7WJ6lmxa1WQ.mp3 — Skipping.
Transcript already exists for: fqB-NcZmqXo.mp3 — Skipping.
Transcript already exists for: eFMeZto6yMg.mp3 — Skipping.
Transcript already exists for: _2IG1CX2y6g.mp3 — Skipping.
Transcript already exists for: it1hcs5S1ks&t=27s.mp3 — Skipping.
Transcript already exists for: a0fllfx_fmg&list=PLkGSnjw5y2U407_1UQQaVVrD13-MFi5ia&index=22.mp3 — Skipping.
Transcript already exists for: kQV6g8Vbbfc.mp3 — Skipping.
Transcript already e

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

transcript_folder = "/content/drive/MyDrive/ServiceNow_Audio_Transcripts/"
all_chunks = []

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

for filename in os.listdir(transcript_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(transcript_folder, filename), "r", encoding="utf-8") as f:
            raw_text = f.read()
            chunks = splitter.split_text(raw_text)
            all_chunks.extend(chunks)

In [5]:
!pip install faiss-cpu sentence-transformers langchain

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store
faiss_store = FAISS.from_texts(all_chunks, embedding=embedding_model)


/tmp/ipython-input-6-2798286620.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [7]:
!pip install -U langchain-openai
from langchain_openai import OpenAI
import os
from google.colab import userdata
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get('OPENAI_API_KEY')
)

In [9]:
from langchain.chains import RetrievalQA
retriever = faiss_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Ask a question
question = "What are the steps to configure incident management in ServiceNow?"
answer = qa_chain.run(question)
print("Answer:", answer)

/tmp/ipython-input-9-18830977.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(question)


Answer: 
1. Set up a ServiceNow instance: The first step is to set up a ServiceNow instance and configure it according to your organization's needs.

2. Define incident management roles: Identify the roles and responsibilities of the team members who will be involved in incident management. This can include incident managers, service desk agents, and other IT staff.

3. Configure incident management settings: In the ServiceNow instance, go to the Incident Management application and configure the settings according to your organization's needs. This includes setting up incident categories, priorities, and SLAs.

4. Create incident templates: Create templates for common incidents to help streamline the incident creation process.

5. Set up incident assignment rules: Configure rules to automatically assign incidents to the appropriate team or individual based on the incident category, priority, or other criteria.

6. Configure notifications: Set up notifications to keep stakeholders infor

In [10]:
question = "How does ServiceNow use AI to improve incident management?"
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  ServiceNow uses AI in their AI Ops solution to assist and automate the work of IT operations. This includes using AI fundamentals like machine learning and pattern recognition to help with incident management and move away from an incident creation culture to an alert-based, actionable mindset. Additionally, ServiceNow's AI-powered service operations enables the use of bots to make decisions and improve incident management processes.


In [11]:
question = "What is predictive intelligence in ServiceNow?"
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  Predictive intelligence in ServiceNow is a feature that requires the creation of three different models to predict three different fields. It is different from task intelligence, which uses a single model and is purpose-built for business users.


In [12]:
question = "Explain how machine learning models are trained in ServiceNow Predictive Intelligence."
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  In ServiceNow Predictive Intelligence, machine learning models are trained by creating three different models to predict three different fields. This requires training the model specifically on ServiceNow workflows, behaviors, and architecture. This is different from task intelligence, where a single model is applicable to multiple fields and is trained using deep learning and long text support models. Additionally, in task intelligence, both the short description and description fields can be used as inputs, which has been shown to provide better results. The number of records used for training in predictive intelligence is also similar to task intelligence.


In [14]:
question = "What AI features are available in the ServiceNow platform?"
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  The ServiceNow platform offers AI features such as AI agents, assisted and autonomous workflows, and the ability to combine data from different sources using AI. These features are native to the platform and do not require customers to learn complex AI tools.
